In [ ]:
!sudo pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pytube import YouTube
def download(url):
    youtubeObject = YouTube(url)
    youtubeObject = youtubeObject.streams.get_lowest_resolution()
    try:
        youtubeObject.download()
    except:
        print("An error has occurred")
    print("Download is completed successfully")

#url = "https://www.youtube.com/watch?v=FJZ-BHBKyos" # car chase
#url = "https://www.youtube.com/watch?v=1gwglom4FeA" # 8 hr nature
#url = "https://www.youtube.com/watch?v=8W1qF7l2A1c" # 6 hr nature
#download(url)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/CS280_final_project/deep_fusion_experiments
#%cd /content/drive/
%ls

/content/drive/MyDrive/CS280_final_project/deep_fusion_experiments
df_vid_aud_out_5.11.mp4         nature_out_5.11aud.mp3  __temp__.mp4
hands_clapping.MOV              nature_out_5.11vid.mp4  vid_to_aud_model.pt
nature_out_5.11aud_and_vid.mp4  nature_short.mp4


In [ ]:
!sudo pip install scikit-video

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import soundfile as sf
import numpy as np
import skvideo.io  
import matplotlib.pyplot as plt

In [ ]:
video_string = "hands_clapping.MOV"
output_string = "hands_out_5.11"

In [ ]:
##NOTE: use vreader and then create a downsampled np array
## try 36x64
resized_vid_arr = []
videodata = skvideo.io.vreader(video_string)  
for v in videodata:
  roi = cv2.resize(v, (36, 64))
  roi = roi.astype("float") / 255.0
  # roi = img_to_array(roi)
  # roi = np.expand_dims(roi, axis=0)
  resized_vid_arr.append(roi) 
resized_vid_arr = np.array(resized_vid_arr)

In [ ]:
from moviepy.video.io.VideoFileClip import AudioFileClip
audioclip = AudioFileClip(video_string)
# audioclip = videoclip.audio
audio_array = audioclip.to_soundarray()

  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



In [ ]:
video_and_audio_arr = []
aud_per_frame = audio_array.shape[0]// resized_vid_arr.shape[0]
for i in range(len(resized_vid_arr)):
  aud_in = audio_array[i*aud_per_frame : (i+1)*aud_per_frame+1]
  vec_in = [resized_vid_arr[i], aud_in]
  video_and_audio_arr.append(vec_in)

In [ ]:
fps = 30
def make_frame(t):
  t = int(t * fps)
  # print(t)
  return video_and_audio_arr[t][0]

from moviepy.video.io.VideoFileClip import VideoClip
myclip = VideoClip(make_frame, duration = 20)

In [ ]:
## concatonate the audio
audio = video_and_audio_arr[0][1]
for i in range(20*40):
  next_aud = video_and_audio_arr[i+1][1]
  audio = np.vstack((audio, next_aud))
print(audio.shape)

(1178271, 2)


In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pydub 

def write(f, sr, x, normalized=True):
    """numpy array to MP3"""
    channels = 2 if (x.ndim == 2 and x.shape[1] == 2) else 1
    if normalized:  # normalized array - each item should be a float in [-1, 1)
        y = np.int16(x * 2 ** 15)
    else:
        y = np.int16(x)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=sr, sample_width=2, channels=channels)
    song.export(f, format="mp3", bitrate="320k")

# sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
#write('test_audio_2.mp3', sr, audio)

*training loop from cs189 hw*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import gc

In [ ]:
class VideoAudioDataset(Dataset):
    def __init__(self, video_frames, audio_frames, num_frames):
        self.video_frames = torch.tensor(video_frames, dtype=torch.float32).permute(0,3,2,1) # Permute to (N, C, H, W)
        self.aud_per_frame = audio_frames.shape[0]// (video_frames.shape[0])
        clip_amount = audio_frames.shape[0] % self.aud_per_frame
        self.audio_frames = torch.tensor(audio_frames[:-clip_amount], dtype=torch.float32).reshape(-1, self.aud_per_frame,audio_frames.shape[1]).permute(0,2,1) # Permute to (N, C, A)
        print(self.audio_frames.shape)
        self.num_frames = num_frames

    def __len__(self):
        return len(self.video_frames)

    def __getitem__(self, idx):
        idx += 1
        if idx < self.num_frames:
          num_zeros_needed = self.num_frames - idx
          vid_zeros = torch.zeros(num_zeros_needed, *self.video_frames[0].shape)
          aud_zeros = torch.zeros(num_zeros_needed, *self.audio_frames[0].shape)
          vid = torch.vstack((vid_zeros, self.video_frames[0:idx])).transpose(0,1)
          aud = torch.vstack((aud_zeros, self.audio_frames[0:idx])).transpose(0,1)
          return (vid, aud)
        return (self.video_frames[idx-self.num_frames:idx].transpose(0,1), self.audio_frames[idx-self.num_frames: (idx)].transpose(0,1))

In [ ]:
print(resized_vid_arr.shape, resized_vid_arr.size)
print(audio_array.size)

(1466, 64, 36, 3) 10132992
4310334


In [ ]:
gc.collect()
train_data = VideoAudioDataset(resized_vid_arr[:int(len(resized_vid_arr)*0.8)], audio_array[:int(len(audio_array)*0.8)], num_frames = 10)
valid_data = VideoAudioDataset(resized_vid_arr[int(len(resized_vid_arr)*0.8):], audio_array[int(len(audio_array)*0.8):], num_frames = 10)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=16, shuffle=True)
#del videodata, audio_array
gc.collect()

torch.Size([1172, 2, 1471])
torch.Size([294, 2, 1466])


0

In [ ]:
for x in train_loader:
  print(x[0].shape)
  print(x[1].shape)
  break

torch.Size([16, 3, 10, 36, 64])
torch.Size([16, 2, 10, 1471])


In [ ]:
class ConvResBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.1, stride=1, kernel_size=3, padding=1):
        super(ConvResBlock3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=True)
        self.norm1 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=True)
        self.norm2 = nn.BatchNorm3d(out_channels)
        self.stride = stride
        
        if in_channels == out_channels:
            self.skip_connection = nn.Identity()
        else:
            self.skip_connection = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm3d(out_channels)
            )
            
    def forward(self, x):
        residual = self.skip_connection(x)
        
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.dropout(out)
        
        out = self.relu(out)
        out = out + residual

        return out

In [ ]:
class ConvResBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.1, stride=1, kernel_size=3, padding=1):
        super(ConvResBlock2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=True)
        self.norm1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=True)
        self.norm2 = nn.BatchNorm2d(out_channels)
        self.stride = stride
        
        if in_channels == out_channels:
            self.skip_connection = nn.Identity()
        else:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        residual = self.skip_connection(x)
        
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.dropout(out)
        
        out = self.relu(out)
        out = out + residual

        return out

In [ ]:
class AudToVidFusion(nn.Module):
    def __init__(self, audio_dim, video_dims, audio_channels, vid_channels, dropout_prob=0.1):
        super(AudToVidFusion, self).__init__()
        self.dim_proj = nn.Linear(audio_dim, video_dims[0] * video_dims[1])
        self.audio_dim = audio_dim
        self.video_dims = video_dims
        self.vid_channels = vid_channels

        if audio_channels == vid_channels:
            self.channel_projection = nn.Identity()
        else:
            self.channel_projection = nn.Sequential(
                nn.Conv2d(audio_channels, vid_channels, kernel_size=1, stride=1, bias=True),
            )
            
    def forward(self, x):
        audio_proj = self.dim_proj(x)
        audio_proj = self.channel_projection(audio_proj)
        proj_reshape = audio_proj.reshape(x.shape[0], self.vid_channels, x.shape[2], self.video_dims[0],self.video_dims[1])

        return proj_reshape

In [ ]:
class VidToAudFusion(nn.Module):
    def __init__(self, audio_dim, video_dims, audio_channels, vid_channels, dropout_prob=0.1):
        super(VidToAudFusion, self).__init__()
        self.dim_proj = nn.Linear(video_dims[0] * video_dims[1], audio_dim)
        self.audio_dim = audio_dim
        self.video_dims = video_dims
        self.vid_channels = vid_channels

        if audio_channels == vid_channels:
            self.channel_projection = nn.Identity()
        else:
            self.channel_projection = nn.Sequential(
                nn.Conv2d(vid_channels, audio_channels, kernel_size=1, stride=1, bias=True),
            )
            
    def forward(self, x):
      #x = x.reshape(x.shape[0], x.shape[1], x.shape[2], -1)
      x = torch.flatten(x, start_dim=3)
      audio_proj = self.dim_proj(x)
      audio_proj = self.channel_projection(audio_proj)

      return audio_proj

In [ ]:
class vid_aud_conv(nn.Module):
  def __init__(self,audio_input_shape, video_input_shape, t=10):
    super(vid_aud_conv, self).__init__()
    self.audio_input_shape = audio_input_shape
    self.video_input_shape = video_input_shape

    self.vid_convs = nn.ModuleList([
        ConvResBlock3D(3, 64),
        ConvResBlock3D(64, 128),
        ConvResBlock3D(128, 256)
    ])

    self.aud_convs = nn.ModuleList([
        ConvResBlock2D(2, 64),
        ConvResBlock2D(64, 128),
        ConvResBlock2D(128, 256)
    ])

    aud_dims_list = self.get_audio_dims(audio_input_shape)
    vid_dims_list = self.get_video_dims(video_input_shape)

    self.aud_to_vid_merges = nn.ModuleList([
        AudToVidFusion(aud_dims_list[0], vid_dims_list[0], 64, 64),
        AudToVidFusion(aud_dims_list[1], vid_dims_list[1], 128, 128),
        AudToVidFusion(aud_dims_list[2], vid_dims_list[2], 256, 256)
    ])

    self.vid_to_aud_merges = nn.ModuleList([
        VidToAudFusion(aud_dims_list[0], vid_dims_list[0], 64, 64),
        VidToAudFusion(aud_dims_list[1], vid_dims_list[1], 128, 128),
        VidToAudFusion(aud_dims_list[2], vid_dims_list[2], 256, 256)
    ])
    self.output_proj = nn.Sequential(
        nn.Conv2d(256, 2, kernel_size=1, stride=1, bias=True),
        nn.Linear(aud_dims_list[2], 100),
        nn.ReLU()
    )

    self.output_layers = nn.Sequential(
        nn.Linear(t*100, 2000),
        nn.ReLU(),
        nn.Linear(2000,2000),
        nn.ReLU(),
        nn.Linear(2000,self.audio_input_shape[-1]),
        nn.Tanh()
    )

  def get_audio_dims(self, audio_input_shape):
    shape_list = []
    test_data = torch.ones(audio_input_shape)
    out = self.aud_convs[0](test_data)
    shape_list.append(out.shape[-1])
    for layer in self.aud_convs[1:]:
        out = layer(out)
        shape_list.append(out.shape[-1])
    return shape_list

  def get_video_dims(self, video_input_shape):
    shape_list = []
    test_data = torch.ones(video_input_shape)
    out = self.vid_convs[0](test_data)
    shape_list.append((out.shape[-2],out.shape[-1]))
    for layer in self.vid_convs[1:]:
        out = layer(out)
        shape_list.append((out.shape[-2],out.shape[-1]))
    return shape_list

  def forward(self, vid, aud):
    for i in range(len(self.vid_convs)):
      #print (i)
      vid_out = self.vid_convs[i](vid)
      aud_out = self.aud_convs[i](aud)

      vid = vid_out + self.aud_to_vid_merges[i](aud_out)
      aud = aud_out + self.vid_to_aud_merges[i](vid_out)

    output_proj = self.output_proj(aud)
    #print(aud.shape)
    #print(output_proj.shape)

    #output_proj_reshaped = output_proj.reshape(aud.shape[0], 2, -1)
    output_flatten = torch.flatten(output_proj, start_dim=2)
    #print(output_flatten.shape)
    output = self.output_layers(output_flatten)

    return output

    #print(vid.shape, aud.shape)





In [ ]:
audio_test_data = torch.ones((1,2,10,1471))
vid_test_data = torch.ones((1,3,10,36,64))

In [ ]:
model = vid_aud_conv(audio_input_shape=audio_test_data.shape, video_input_shape=vid_test_data.shape).cuda()

In [ ]:
from tqdm.notebook import tqdm, trange

def loss_fn(output, target):
  # v_out = torch.abs(output.cuda() - target.cuda())
  # mean = torch.mean(v_out)
  # # print('mean = ', mean)

  loss = nn.CrossEntropyLoss()
  result = loss(output.cpu(), target.cpu())
  return result

def run_training_loop(model, train_loader, valid_loader, n_epochs=10, lr=1e-3, model_file="vid_to_aud_model.pt"):

    # We could write our training procedure manually and directly index the `Dataset` objects,
    # but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:
    # train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

    # Choose Adam as the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Use the cross entropy loss function
    #loss_fn = nn.CrossEntropyLoss()

    # store metrics
    train_loss_history = np.zeros([n_epochs, 1])
    valid_accuracy_history = np.zeros([n_epochs, 1])
    valid_loss_history = np.zeros([n_epochs, 1])
    best_metric = np.inf

    for epoch in trange(n_epochs, desc="training", unit="epoch"):

        # Some layers, such as Dropout, behave differently during training
        model.train()

        train_loss = 0
        
        batch_iterator = tqdm(train_loader, desc="epoch {}".format(epoch + 1), unit="batch", total=len(train_loader))
        for batch_idx, data in enumerate(batch_iterator):
            #print(data[0].shape)
            #print(data[1].shape)
            vid, aud = data
            target = aud[:,:,-1].clone().detach()
            aud[:,:,-1] *= 0
            vid = vid.cuda()
            aud = aud.cuda()

            # Forward pass
            output = model(vid, aud)

            # Calculate loss
            loss = loss_fn(output.cpu(), target.cpu())
            # print(loss)
            train_loss += loss.item()

            # Erase accumulated gradients
            optimizer.zero_grad()

            # Backward pass
            loss.backward()
            
            # Weight update
            optimizer.step()

            batch_iterator.set_postfix({"Train Loss:": train_loss / len(vid) * batch_idx})

        train_loss_history[epoch] = train_loss / len(train_loader.dataset)

        # Track loss each epoch
        print('Train Epoch: %d  Average loss: %.4f' %
              (epoch + 1,  train_loss_history[epoch]))

        # Putting layers like Dropout into evaluation mode
        model.eval()

        valid_loss = 0
        correct = 0

        # Turning off automatic differentiation
        with torch.no_grad():
            for data in valid_loader:
                vid, aud = data
                print('aud.shape = ', aud.shape)
                if aud.shape[3] < audio_test_data.shape[3]:
                  zeros = torch.zeros(aud.shape[0], aud.shape[1], aud.shape[2], audio_test_data.shape[3])
                  print('clipped zeros shape',zeros[:audio_test_data.shape[0], :audio_test_data.shape[1], :audio_test_data.shape[2], :audio_test_data.shape[3]].shape)
                  zeros[:, :, :, :aud.shape[3]] = aud
                  aud = zeros
                  print(aud.shape)
                target = aud[:,:,-1].clone().detach()
                aud[:,:,-1] *= 0
                vid = vid.cuda()
                aud = aud.cuda()
                output = model(vid, aud)
                output = output.cpu()
                valid_loss += loss_fn(output, target).item()  # Sum up batch loss
                vid.cpu()
                aud.cpu()
                # pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
                # correct += pred.eq(target.view_as(pred)).sum().item()
            if valid_loss < best_metric:
                state_dict = model.state_dict()
                print(
                    "Obtained a new best validation metric of {:.3f}, saving model "
                    "checkpoint to {}...".format(valid_loss, model_file))
                torch.save(state_dict, model_file)
                best_metric = valid_loss

        valid_loss_history[epoch] = valid_loss / len(valid_loader.dataset)
        # valid_accuracy_history[epoch] = correct / len(valid_loader.dataset)

        # print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %
        #       (valid_loss_history[epoch], correct, len(valid_loader.dataset)))
        print('Valid loss = ', valid_loss_history[epoch], ' epoch = ', epoch)
        gc.collect()
    
    return model, train_loss_history, valid_loss_history

In [ ]:
torch.autograd.set_detect_anomaly(True)

run_training_loop(model, train_loader, valid_loader, n_epochs = 30, lr=1e-3)

training:   0%|          | 0/30 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/74 [00:00<?, ?batch/s]

Train Epoch: 1  Average loss: -0.0000
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 

epoch 2:   0%|          | 0/74 [00:00<?, ?batch/s]

Train Epoch: 2  Average loss: -0.0000
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 

...

epoch 29:   0%|          | 0/74 [00:00<?, ?batch/s]

Train Epoch: 29  Average loss: -0.0000
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16,

epoch 30:   0%|          | 0/74 [00:00<?, ?batch/s]

Train Epoch: 30  Average loss: -0.0000
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16, 2, 10, 1471])
aud.shape =  torch.Size([16, 2, 10, 1466])
clipped zeros shape torch.Size([1, 2, 10, 1471])
torch.Size([16,

(vid_aud_conv(
   (vid_convs): ModuleList(
     (0): ConvResBlock3D(
       (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU()
       (dropout): Dropout(p=0.1, inplace=False)
       (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (norm2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (skip_connection): Sequential(
         (0): Conv3d(3, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1))
         (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (1): ConvResBlock3D(
       (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (norm1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU()
       (dropout): Dropout(p=0.1, i

In [ ]:
class VideoOnlyDataset(Dataset):
    def __init__(self, video_frames, num_frames):
        self.video_frames = torch.tensor(video_frames, dtype=torch.float32).permute(0,3,2,1) # Permute to (N, C, H, W)
        self.num_frames = num_frames

    def __len__(self):
        return len(self.video_frames)

    def __getitem__(self, idx):
        idx += 1
        if idx < self.num_frames:
          num_zeros_needed = self.num_frames - idx
          vid_zeros = torch.zeros(num_zeros_needed, *self.video_frames[0].shape)
          vid = torch.vstack((vid_zeros, self.video_frames[0:idx])).transpose(0,1)
          return vid
        #vid shape example torch.Size([32, 3, 10, 36, 64])
        # aud shape example torch.Size([32, 2, 14710])
        vid = self.video_frames[idx-self.num_frames:idx].transpose(0,1)
        # print('idx = ', idx, ' aud.size = ', aud.shape)
        return vid

In [ ]:
## get the data again
#NOTE: use vreader and then create a downsampled np array
resized_vid_arr = []
larger_vid_arr = []
videodata = skvideo.io.vreader(video_string)  

count = 0 
for v in videodata:
  count += 1
  if count % 3 == 0:
    roi = cv2.resize(v, (36, 64))
    roi = roi.astype("float") / 255.0
    resized_vid_arr.append(roi) 
    larger_vid_arr.append(v)
resized_vid_arr = np.array(resized_vid_arr)

# from moviepy.video.io.VideoFileClip import AudioFileClip
# audioclip = AudioFileClip(video_string, fps=8000)
# # audioclip = videoclip.audio
# audio_array = audioclip.to_soundarray()

from moviepy.video.io.VideoFileClip import AudioFileClip
audioclip = AudioFileClip(video_string)
# audioclip = videoclip.audio
audio_array = audioclip.to_soundarray()

  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



In [ ]:
### generate audio for video from model
start_index = int(len(resized_vid_arr)*0.8)
audio_list = []
vid_test_shape = (1,3,10,36,64)
zero_frame = torch.ones((vid_test_shape[4], vid_test_shape[4]))

aud_input_arr = torch.zeros((1,2,10,1471))

input_vid = resized_vid_arr[start_index:]
audio_zeros = torch.zeros(1471*len(input_vid),2)
print(input_vid.shape, audio_zeros.shape)
gen_data = VideoAudioDataset(input_vid, audio_array, num_frames = 10)
gen_loader = DataLoader(gen_data, batch_size=1, shuffle=False)

video_list = []

i = 0
for vid, _ in gen_loader:
  vid = vid.cuda()
  with torch.no_grad():
    # print(vid.shape)
    # print(aud_input_arr.shape)
    audio_output = model(vid.cuda(), aud_input_arr.cuda()).cpu()
  video_list.append(larger_vid_arr[start_index + i])
  audio_list.append(audio_output.squeeze().cpu().numpy())
  aud_input_arr = aud_input_arr[:, :, 1:, :]
  aud_input_arr = torch.cat((aud_input_arr, audio_output.unsqueeze(2)), 2)
  i += 1

video_np = np.array(video_list)
print()
audio_np_pre = np.array(audio_list)
# print(audio_np.shape)
# audio_np = np.swapaxes(audio_np, 1,0)
# print(np.array(audio_list).shape)
audio_np = audio_np_pre[0]
print('audio np initial shape = ', audio_np.shape)
for i in range(audio_np_pre.shape[0] - 1):
  i = i+1
  audio_np = np.concatenate((audio_np, audio_np_pre[i]), axis = 1)

print('audio np post shape = ', audio_np.shape)

# audio_np = np.array(audio_list).transpose(1,0)
# fps = 10
# sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
# write('deep_fusion_out5.11.mp3', sr, audio_np)

(98, 64, 36, 3) torch.Size([144158, 2])
torch.Size([98, 2, 21991])

audio np initial shape =  (2, 1471)
audio np post shape =  (2, 144158)


In [ ]:
video_np.shape

(98, 1080, 1920, 3)

In [ ]:
fps = 10
def make_frame(t):
  t = int(t * fps)
  # print(t)
  return video_np[t]

from moviepy.video.io.VideoFileClip import VideoClip
myclip = VideoClip(make_frame, duration = 3)
vid_out_path = output_string + 'vid.mp4'
myclip.write_videofile(vid_out_path, fps = 10)

Moviepy - Building video hands_out_5.11vid.mp4.
Moviepy - Writing video hands_out_5.11vid.mp4



Moviepy - Done !
Moviepy - video ready hands_out_5.11vid.mp4


In [ ]:
fps = 30
sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
print(sr, audio_np.shape)
aud_out_path = output_string + 'aud.mp3'
write(aud_out_path, sr, audio_np)

132480 (2, 144158)


In [ ]:
from moviepy.editor import *

clip = VideoFileClip(vid_out_path)
  
clip = clip.subclip(0, 3)
  
# loading audio file
audioclip = AudioFileClip(aud_out_path)
  
# adding audio to the video clip
videoclip = clip.set_audio(audioclip)

vid_p_aud_out_path = output_string + 'aud_and_vid.mp4'
videoclip.write_videofile(vid_p_aud_out_path, fps = 10)
  
# showing video clip
videoclip.ipython_display()

Moviepy - Building video hands_out_5.11aud_and_vid.mp4.
MoviePy - Writing audio in hands_out_5.11aud_and_vidTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video hands_out_5.11aud_and_vid.mp4



Moviepy - Done !
Moviepy - video ready hands_out_5.11aud_and_vid.mp4
Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
